In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anemia-types-classification/diagnosed_cbc_data_v4.csv


In [2]:
# Quick view of dataset content and structure.
df= pd.read_csv('/kaggle/input/anemia-types-classification/diagnosed_cbc_data_v4.csv')
df.head()

,WBC,LYMp,NEUTp,LYMn,NEUTn,RBC,HGB,HCT,MCV,MCH,MCHC,PLT,PDW,PCT,Diagnosis
0,10.0,43.2,50.1,4.3,5.0,2.77,7.3,24.2,87.7,26.3,30.1,189.0,12.5,0.17,Normocytic hypochromic anemia
1,10.0,42.4,52.3,4.2,5.3,2.84,7.3,25.0,88.2,25.7,20.2,180.0,12.5,0.16,Normocytic hypochromic anemia
2,7.2,30.7,60.7,2.2,4.4,3.97,9.0,30.5,77.0,22.6,29.5,148.0,14.3,0.14,Iron deficiency anemia
3,6.0,30.2,63.5,1.8,3.8,4.22,3.8,32.8,77.9,23.2,29.8,143.0,11.3,0.12,Iron deficiency anemia
4,4.2,39.1,53.7,1.6,2.3,3.93,0.4,316.0,80.6,23.9,29.7,236.0,12.8,0.22,Normocytic hypochromic anemia


In [3]:
# Import necessary libraries for preprocessing, splitting data, and building the neural network.
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [4]:
y = df['Diagnosis']  # Target variable indicating the diagnosis.
x = df.drop('Diagnosis', axis=1)  # Features used for prediction.

In [5]:
# Displays the target variable `y` to check its values and ensure correctness.
print(y) # y

0       Normocytic hypochromic anemia
1       Normocytic hypochromic anemia
2              Iron deficiency anemia
3              Iron deficiency anemia
4       Normocytic hypochromic anemia
                    ...              
1276                          Healthy
1277                          Healthy
1278                          Healthy
1279                          Healthy
1280                          Healthy
Name: Diagnosis, Length: 1281, dtype: object


In [6]:
# Encodes the target variable `y` into numeric values using LabelEncoder.
# This converts the categorical labels into integers for model compatibility.
le=LabelEncoder()
y=le.fit_transform(y)

In [7]:
y

array([5, 5, 1, ..., 0, 0, 0])

In [8]:
# Converts the integer-encoded labels into a one-hot encoded format.
# This is necessary for multi-class classification tasks in neural networks.
y=to_categorical(y)

In [9]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [10]:
y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [11]:
# Splits the dataset into training and testing sets.
# 75% of the data is used for training, and 25% is used for testing.
# random_state ensures reproducibility of the split.
x_train, x_test, y_train, y_test=train_test_split(x,y,train_size=0.75,random_state=43)

In [12]:
# Initializes a Sequential neural network model.
# Adds a dense layer with 64 units and ReLU activation, where input_dim specifies the number of features in the dataset.
# Adds two more dense layers with 32 units and ReLU activation to learn non-linear patterns.
# The final layer has 9 units, corresponding to the 9 categories in the target variable, and uses softmax activation for multi-class classification.

model=Sequential()

model.add(Dense(64,activation="relu",input_dim=len(x.columns))) 
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))

model.add(Dense(9,activation="softmax"))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Compiles the model using the Adam optimizer, which adapts learning rates during training.
# Uses categorical crossentropy as the loss function for multi-class classification.
# Tracks the accuracy metric to evaluate the model's performance during training and testing.
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
# Trains the model on the training data (x_train, y_train).
# The training will run for 128 epochs, with a batch size of 32.
# Uses 24% of the training data for validation during training (validation_split=0.24).
model.fit(x_train,y_train,epochs =128,batch_size=32,validation_split=0.24)

Epoch 1/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.1105 - loss: 11.3497 - val_accuracy: 0.2641 - val_loss: 2.3064
Epoch 2/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3197 - loss: 2.2814 - val_accuracy: 0.4719 - val_loss: 1.9361
Epoch 3/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3410 - loss: 1.8380 - val_accuracy: 0.3550 - val_loss: 1.8691
Epoch 4/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3276 - loss: 1.6876 - val_accuracy: 0.4935 - val_loss: 1.8757
Epoch 5/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4071 - loss: 1.6042 - val_accuracy: 0.4545 - val_loss: 1.8807
Epoch 6/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4073 - loss: 1.5463 - val_accuracy: 0.4416 - val_loss: 1.8968
Epoch 7/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4294 - loss: 1.4859 - val_accuracy: 0.5065 - val_loss: 1.8429
Epoch 8/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4355 - loss: 1.4876 - val_accuracy: 0.4113 -

In [15]:
# Create an array with new input data (e.g., for a new patient).
# The data includes various medical measurements.
new_data = np.array([[9.4,34,58,2.8,4.3,3.3,8.8,32,78,24,26,155,15,0.17]])

# Use the trained model to predict the class (diagnosis) for the new input data.
prediction = model.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


In [16]:
# Print the predicted class. The model will return probabilities, 
# and you can use the `argmax` function to get the class with the highest probability.
predicted_class = np.argmax(prediction)

In [17]:
# Print the predicted class (diagnosis).
print("Predicted Diagnosis Class:", predicted_class)

Predicted Diagnosis Class: 5


In [18]:
# Convert the encoded class label (e.g., 5) back to its original category using the LabelEncoder.
original_class = le.inverse_transform([5])

# Print the original class.
print("Original Class:", original_class)

Original Class: ['Normocytic hypochromic anemia']
